In this notebook, we show a simple example for doing Exact inference in Bayesian Networks using pgmpy. We will be using the Asia network (http://www.bnlearn.com/bnrepository/#asia) for this example.

### Step 1: Define the model.

In [1]:
# Fetch the asia model from the bnlearn repository

from pgmpy.utils import get_example_model

asia_model = get_example_model('asia')

In [2]:
print("Nodes: ", asia_model.nodes())
print("Edges: ", asia_model.edges())
asia_model.get_cpds()

Nodes:  ['asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp']
Edges:  [('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')]


[<TabularCPD representing P(asia:2) at 0x7f7ed0cb09d0>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x7f7ed0cb0c10>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x7f7ed0c57a00>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x7f7ed0c57880>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x7f7ed0c57f70>,
 <TabularCPD representing P(smoke:2) at 0x7f7ed0c57f10>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x7f7ed0c57e80>,
 <TabularCPD representing P(xray:2 | either:2) at 0x7f7ed0c57c70>]

If you would like to create a model from scratch, please refer to the Creating Bayesian Networks notebook: https://github.com/pgmpy/pgmpy/blob/dev/examples/Creating%20a%20Bayesian%20Network.ipynb

### Step 2: Initialize the inference class

Currently, pgmpy support two algorithms for inference: 1. Variable Elimination and, 2. Belief Propagation. Both of these are exact inferece algorithms. The following example uses `VariableElimination` but `BeliefPropagation` has an identifcal API, so all the methods show below would also work for `BeliefPropagation`.

In [3]:
# Initializing the VariableElimination class

from pgmpy.inference import VariableElimination
asia_infer = VariableElimination(asia_model)

### Step 3: Doing Inference

In [4]:
# Computing the probability of bronc given smoke=no.
q = asia_infer.query(variables=['bronc'], evidence={'smoke': 'no'})
print(q)

# Computing the joint probability of bronc and asia given smoke=yes
q = asia_infer.query(variables=['bronc', 'asia'], evidence={'smoke': 'yes'})
print(q)

# Computing the probabilities (not joint) of bronc and asia given smoke=no
q = asia_infer.query(variables=['bronc', 'asia'], evidence={'smoke': 'no'}, joint=False)
for factor in q.values():
    print(factor)

Eliminating: xray: 100%|██████████| 5/5 [00:00<00:00, 392.84it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+
+------------+-----------+-------------------+
| bronc      | asia      |   phi(bronc,asia) |
+============+===========+===================+
| bronc(yes) | asia(yes) |            0.0060 |
+------------+-----------+-------------------+
| bronc(yes) | asia(no)  |            0.5940 |
+------------+-----------+-------------------+
| bronc(no)  | asia(yes) |            0.0040 |
+------------+-----------+-------------------+
| bronc(no)  | asia(no)  |            0.3960 |
+------------+-----------+-------------------+
+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+
+-----------+-------------+
| asia      |   phi(asia) |
+======

In [5]:
# Computing the MAP of bronc given smoke=no.
q = asia_infer.map_query(variables=['bronc'], evidence={'smoke': 'no'})
print(q)

# Computing the MAP of bronc and asia given smoke=yes
q = asia_infer.map_query(variables=['bronc', 'asia'], evidence={'smoke': 'yes'})
print(q)

Eliminating: xray: 100%|██████████| 5/5 [00:00<00:00, 558.72it/s]

{'bronc': 'no'}
{'bronc': 'yes', 'asia': 'no'}


### Step 4: Troubleshooting for slow inference

In the case of large models, or models in which variables have a lot of states, inference can be quite slow. Some of the ways to deal with it are:

1. Reduce the number of states of variables by combining states together.
2. Try a different elimination order by specifying `elimination_order` argument. Possible options are: MinFill, MinNeighbors, MinWeight, WeightedMinFill. 
3. Try a custom elimination order: The implemented heuristics for computing the elimination order might not be efficient in every case. If you can think of a more efficient order, you can also pass it as a list to the `elimination_order` argument.
4. If it is still too slow, try using approximate inference using sampling algorithms.